In [1]:
import utils
import keras
import model01
import datasets
import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

Using TensorFlow backend.


In [2]:
class EnsembleModel():
    def __init(self, models):
        self.models = models
        
    def predict(self, X, **kwargs):
        preds = [ model.predict(X, **kwargs) for model in self.models]
        predicted_classes = [utils.prediction_to_category2(prediction) for prediction in preds]
        print "preds", preds[:5]
        print "pred classes", predicted_classes[:5]
        

In [4]:
def model_name_from_weights_file(wf):
    return "-".join(wf[9:-5].split('-')[:-2])


def report_performance(model_details, 
                       X_test, Y_test, prices_test, title, 
                       learning_curve=False, ylims=(0.68,0.70), **kwargs):
    
    name, weight_file = model_details
    weight_file = './output/' + weight_file
    model = keras.models.load_model(weight_file)
    
    
    Y_test_pred = model.predict(X_test.as_matrix(), batch_size=1024)
    print name
    metrics.performance_report(title,  prices_test,  lookahead, Y_test, Y_test_pred, **kwargs)
    if learning_curve:
        pc = model01.ProgressCallback.load(model_name_from_weights_file(weight_file))
        plot_learning_curve(pc, name, ylims)
        
    return model

In [9]:
def load_model(model_details):
    name, weight_file = model_details
    weight_file = './output/' + weight_file
    model = keras.models.load_model(weight_file)
    return model

In [10]:
BEST_MODELS = [
    ['100 dropout=0', 'MLPModel01_20170423_1307_EURUSD_DS3_20092014_LA1_F99_C2_L100_DO0-1000-0.6945.hdf5'],
    ['100x100 dropout=0','MLPModel01_20170423_1448_EURUSD_DS3_20092014_LA1_F99_C2_L100_100_DO0-1000-0.6928.hdf5'],
    ['500 dropout=0','MLPModel01_20170423_1615_EURUSD_DS3_20092014_LA1_F99_C2_L500_DO0-1000-0.6949.hdf5'],
    ['500x500 dropout=0 epoch 1000', 'MLPModel01_20170423_1800_EURUSD_DS3_20092014_LA1_F99_C2_L500_500_DO0-1000-0.7095.hdf5'],
    ['500x500 dropout=0.5','MLPModel01_20170423_2154_EURUSD_DS3_20092014_LA1_F99_C2_L500_500_DO0.5-1000-0.6914.hdf5'],
    ['100x100x100 dropout=0.5', 'MLPModel01_20170418_1151_EURUSD_DS3_20092014_LA1_F99_C2_L100_100_100_DO0.5-1005-0.6913.hdf5'],
    ['500x500x500 dropout=0',   'MLPModel01_20170423_1051_EURUSD_DS3_20092014_LA1_F99_C2_L500_500_500_DO0-015-0.6941.hdf5'],
    ['500x500x500 dropout=0.5', 'MLPModel01_20170424_0713_EURUSD_DS3_20092014_LA1_F99_C2_L500_500_500_DO0.5-2000-0.6913.hdf5'],
    ['500x500x500 dropout=0.5 epoch 1970', 'MLPModel01_20170424_0713_EURUSD_DS3_20092014_LA1_F99_C2_L500_500_500_DO0.5-1970-0.6912.hdf5'],
    

    ['100x100x100 dropout=0 epoch 20',  'MLPModel01_20170418_1912_EURUSD_DS3_20092014_LA1_F99_C2_L100_100_100_DO0-020-0.6932.hdf5'],
    ['100x100x100 dropout=0 epoch 115', 'MLPModel01_20170418_1912_EURUSD_DS3_20092014_LA1_F99_C2_L100_100_100_DO0-115-0.6934.hdf5'],
    ['500x500 dropout=0 epoch 165', 'MLPModel01_20170423_1800_EURUSD_DS3_20092014_LA1_F99_C2_L500_500_DO0-165-0.6954.hdf5'],
    ['32x32x32x32 dropout=0.5', 'MLPModel01_20170423_1239_EURUSD_DS3_20092014_LA1_F99_C2_L32_32_32_32_DO0.5-455-0.6931.hdf5'],
]

In [11]:
# load test dataset
lookahead=1
dataset = 'DS3'
window=25
sym='EURUSD'

year=2015
test_title = sym + " " + str(year)
X_test, Y_test, prices_test = datasets.load(datasets.filename(dataset, lookahead, window, sym, year))

In [134]:
from sklearn.externals import joblib
model00 = joblib.load('./output/gnb_EURUSD_DS3_20092014.pkl') 

model01 = load_model(BEST_MODELS[0])
model02 = load_model(BEST_MODELS[1])
model03 = load_model(BEST_MODELS[2])


In [149]:
models = [ load_model(BEST_MODELS[i]) for i in [4,9] ] + [model00]
N_SAMPLES = 15000

act = np.matrix(Y_test.head(N_SAMPLES).ravel())
comp = [ utils.prediction_to_category2(model.predict(X_test.head(N_SAMPLES).as_matrix())).ravel() for model in models]
print [ (p == act).sum()*1.0/N_SAMPLES for p in comp ]

pred = utils.prediction_to_category2(np.matrix(comp).sum(axis=0)/len(models))*1



(pred == act).sum() *1.0/ N_SAMPLES

[0.5158666666666667, 0.52373333333333338, 0.50980000000000003]


0.50480000000000003